In [13]:
import pandas as pd
import os
from datetime import datetime
import warnings

# 抑制特定类型的警告
with warnings.catch_warnings():
    warnings.simplefilter("ignore", category=UserWarning)

# 打开文件A
file_a_path = r'C:\Users\05294\Downloads\外贸信托-盈享致远-2024-01-07.xlsx'
file_a = pd.read_excel(file_a_path)

# 删除资产代码列中字符串开头不是NHG和ZHG开头的行
file_a = file_a[file_a['资产代码'].str.startswith('NHG') | file_a['资产代码'].str.startswith('ZHG')]

# 定义下载目录
download_dir = r'C:\Users\05294\Downloads'

# 获取用户输入的日期
date_input = input("请输入日期（格式如20240107）：")
date = datetime.strptime(date_input, '%Y%m%d')

# 遍历所属资管计划名称列去打开对应的excel文件
plan_files = {}
for index, row in file_a.iterrows():
    plan_name = row['所属资管计划名称']
    date_str = date.strftime('%Y%m%d')  # 将日期格式化为20240107格式
    plan_file_name = f"{plan_name}-{date_str}.xlsx"
    plan_file_path = os.path.join(download_dir, plan_file_name)
    print(plan_file_path)
    if os.path.exists(plan_file_path):
        if plan_file_path not in plan_files:
            plan_files[plan_file_path] = pd.read_excel(plan_file_path,header=3)

        # 获取对应的计划文件
        plan_file = plan_files[plan_file_path]
        print("列名：", plan_file.columns.tolist())
        # 检查'科目名称'列是否存在于计划文件中
        if '科目名称' not in plan_file.columns:
            print(f"文件 {plan_file_path} 中不存在'科目名称'列")
            continue
        # 逆回购
        if row['资产名称'] == '逆回购':
            subject_name = '买入返售金融资产'
            cost_row = plan_file[plan_file['科目名称'] == subject_name]
            if not cost_row.empty:
                file_a.at[index, '全价市值（元）'] = cost_row['成本'].values[0]

        # 正回购
        if row['资产名称'] == '正回购':
            subject_name = '卖出回购证券款'
            cost_row = plan_file[plan_file['科目名称'] == subject_name]
            if not cost_row.empty:
                cost_value = cost_row['成本'].values[0]
                total_forward = file_a[file_a['资产名称'] == '正回购']['全价市值（元）'].sum()
                if total_forward != cost_value:
                    for forward_index, forward_row in file_a[file_a['资产名称'] == '正回购'].iterrows():
                        if '131809' in forward_row['资产代码']:
                            subject_name_deep = '深交所卖出回购证券'
                            cost_row_deep = plan_file[plan_file['科目名称'] == subject_name_deep]
                            if not cost_row_deep.empty:
                                file_a.at[forward_index, '全价市值（元）'] = cost_row_deep['成本'].values[0]
                        elif '204004' in forward_row['资产代码']:
                            subject_name_sh = '上交所卖出回购证券'
                            cost_row_sh = plan_file[plan_file['科目名称'] == subject_name_sh]
                            if not cost_row_sh.empty:
                                file_a.at[forward_index, '全价市值（元）'] = cost_row_sh['成本'].values[0]

# 保存修改后的文件A
file_a.to_excel(file_a_path, index=False)

C:\Users\05294\Downloads\外贸信托-盈享致远1号集合资金信托计划-20240107.xlsx
列名： ['科目代码', '科目名称', '数量', '单位成本', '成本', '成本占净值%', '成本占总资产%', '市价', '市值', '市值占净值%', '市值占总资产%', '估值增值', '停牌信息']
C:\Users\05294\Downloads\外贸信托-盈享致远1号集合资金信托计划-20240107.xlsx
列名： ['科目代码', '科目名称', '数量', '单位成本', '成本', '成本占净值%', '成本占总资产%', '市价', '市值', '市值占净值%', '市值占总资产%', '估值增值', '停牌信息']
C:\Users\05294\Downloads\外贸信托-盈享致远2号集合资金信托计划-20240107.xlsx
列名： ['科目代码', '科目名称', '数量', '单位成本', '成本', '成本占净值%', '成本占总资产%', '市价', '市值', '市值占净值%', '市值占总资产%', '估值增值', '停牌信息']
C:\Users\05294\Downloads\外贸信托-盈享致远3号集合资金信托计划-20240107.xlsx
列名： ['科目代码', '科目名称', '数量', '单位成本', '成本', '成本占净值%', '成本占总资产%', '市价', '市值', '市值占净值%', '市值占总资产%', '估值增值', '停牌信息']
C:\Users\05294\Downloads\外贸信托-盈享致远3号集合资金信托计划-20240107.xlsx
列名： ['科目代码', '科目名称', '数量', '单位成本', '成本', '成本占净值%', '成本占总资产%', '市价', '市值', '市值占净值%', '市值占总资产%', '估值增值', '停牌信息']


D:\anaconda3\Lib\site-packages\openpyxl\styles\stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
D:\anaconda3\Lib\site-packages\openpyxl\styles\stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
D:\anaconda3\Lib\site-packages\openpyxl\styles\stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
